In [ ]:
# default_exp dashboards

# dashboards
> Supplies dashboards to investigate datasets and training results. Dashboards are defined as classes, to show the dashboard use the .show() function on an dashboard instance.

In [ ]:
#export
from typing import Union, Optional, List
from abc import abstractmethod, ABC
from math import ceil, floor

from bokeh.plotting import show, output_notebook
from bokeh.models.widgets import DataTable, TableColumn, HTMLTemplateFormatter
from bokeh.models import ColumnDataSource

import panel as pn
import panel.widgets as pnw
import numpy as np
import pandas as pd

from icevision_dashboards.core.dashboards import *
from icevision_dashboards.plotting import *
from icevision_dashboards.core.data import *
from icevision_dashboards.data import *

In [ ]:
#hide
pn.extension()

## Test data setup

In [ ]:
import icedata

In [ ]:
test_data_dir = icedata.fridge.load_data()
test_class_map = icedata.fridge.class_map()
test_parser = icedata.fridge.parser(test_data_dir, test_class_map)
test_train_records, test_valid_records = test_parser.parse()
test_valid_record_dataset = BboxRecordDataset(test_valid_records, test_class_map)
test_train_record_dataset = BboxRecordDataset(test_train_records, test_class_map)
test_record_dataset_no_class_map = BboxRecordDataset(test_train_records)

In [ ]:
#export
class ObjectDetectionDatasetOverview(DatasetOverview):
    DESCRIPTOR_DATA = "data"
    DESCRIPTOR_STATS_DATASET = "stats_dataset"
    DESCRIPTOR_STATS_IMAGES = "stats_image"
    DESCRIPTOR_STATS_ANNOTATIONS = "stats_class"
    
    # change these 
    IMAGE_IDENTIFIER_COL = "filepath"
    ANNOTATON_LABEL_COL = "label"
    OBJECTS_PER_IMAGE_COL = "num_annotations"
    AREA_COL = "area"
    
    def _generate_datset_stats_tab(self):
        dataset_overview_table = table_from_dataframe(getattr(self.dataset, self.DESCRIPTOR_STATS_DATASET), width=self.width, height=self.height//7)
        images_overview_table = table_from_dataframe(getattr(self.dataset, self.DESCRIPTOR_STATS_IMAGES), width=self.width, height=self.height//7)
        classes_overview_table = table_from_dataframe(getattr(self.dataset, self.DESCRIPTOR_STATS_ANNOTATIONS), width=self.width, height=self.height//4)
        
        class_occurances = self.dataset.data.groupby("label").count()["id"]
        class_occurance_barplot = barplot(counts=class_occurances.values, values=np.array(class_occurances.index), bar_type="vertical", height=(self.height//5)*2)

        return pn.Column("<b>Dataset stats</b>", dataset_overview_table, "<b>Image stats</b>", images_overview_table, "<b>Class stats</b>", classes_overview_table, pn.Row(class_occurance_barplot, align="center"))
    
    def _generate_annotations_tab(self):
        plot_size = floor(min(self.height, self.width)*0.45)
        # mixing of classes
        mixing_matrix_classes_in_images = utils.calculate_mixing_matrix(getattr(self.dataset, self.DESCRIPTOR_DATA), self.IMAGE_IDENTIFIER_COL, self.ANNOTATON_LABEL_COL)
        self.class_mixing_matrix_plot = pn.Column("<b>Class mixing</b>", heatmap(mixing_matrix_classes_in_images, "row_name", "col_name", "values", width=plot_size, height=plot_size), height=self.height)
        # number of object per image, stacked hist
        self.classes_for_objects_per_image_stacked_hist = pn.Column(
            "<b>Objects per Image</b>", 
            stacked_hist(getattr(self.dataset, self.DESCRIPTOR_DATA), self.OBJECTS_PER_IMAGE_COL, self.ANNOTATON_LABEL_COL, "Objects per Image", width=plot_size, height=plot_size)
        )
        # categorical overview
        self.categorical_2d_histogram = categorical_2d_histogram_with_gui(
            getattr(self.dataset, self.DESCRIPTOR_DATA),
            category_cols=["label", "num_annotations", "width", "height"],
            hist_cols=["num_annotations", "area", "area_normalized", "bbox_ratio", "bbox_xmin", "bbox_xmax", "bbox_ymin", "bbox_ymax", "width", "height"])
        # ratio distribution
        grid =  pn.GridSpec(ncols=2,nrows=2, width=self.width, height=self.height, align="center")
        grid[0,0] = self.class_mixing_matrix_plot
        grid[1,0] = self.classes_for_objects_per_image_stacked_hist
        grid[:,1] = pn.Column(self.categorical_2d_histogram, align="center")
        return grid 
    
    def _generate_gallery_tab(self):
        return pn.Column(Gallery(self.dataset, "data", "filepath", ["num_annotations", "width", "height", "label", "area", "bbox_ratio", "bbox_width", "bbox_height"], height=self.height).show(), align="center", sizing_mode="stretch_both")
    
    def build_gui(self):
        dataset_tab = super()._generate_dataset_tab()
        dataset_stats_tab = self._generate_datset_stats_tab()
        annotations_tab = self._generate_annotations_tab()
        gallery_tab = self._generate_gallery_tab()
        self.gui = pn.Tabs(("Dataset stats overview", dataset_stats_tab), ("Annotations overivew", annotations_tab), ("Gallery", gallery_tab), ("Dataset overview", dataset_tab), align="start")

In [ ]:
test_object_detection_overview = ObjectDetectionDatasetOverview(test_valid_record_dataset, width=1500, height=900)
test_object_detection_overview.show()

In [ ]:
#export
class ObjectDetectionDatasetComparison(DatasetComparison):
    DESCRIPTOR_DATA = "data"
    DESCRIPTOR_STATS_DATASET = "stats_dataset"
    DESCRIPTOR_STATS_IMAGES = "stats_image"
    DESCRIPTOR_STATS_ANNOTATIONS = "stats_class"
    
    # change these 
    IMAGE_IDENTIFIER_COL = "filepath"
    ANNOTATON_LABEL_COL = "label"
    OBJECTS_PER_IMAGE_COL = "num_annotations"
    AREA_COL = "area"
    
    def _generate_dataset_tab(self):
        overview_table = table_from_dataframe(self._get_descriptor_for_all_datasets(self.DESCRIPTOR_DATA), width=floor(self.width/2), height=self.height)
        return pn.Row(*overview_table)
    
    def _generate_datset_stats_tab(self):
        dataset_overview_table = table_from_dataframe(self._get_descriptor_for_all_datasets(self.DESCRIPTOR_STATS_DATASET), width=floor(self.width/2), height=self.height//7)
        images_overview_table = table_from_dataframe(self._get_descriptor_for_all_datasets(self.DESCRIPTOR_STATS_IMAGES), width=floor(self.width/2), height=self.height//7)
        classes_overview_table = table_from_dataframe(self._get_descriptor_for_all_datasets(self.DESCRIPTOR_STATS_ANNOTATIONS), width=floor(self.width/2), height=self.height//4)
        
        class_occurances_values = [dataset.data.groupby("label").count()["id"].values for dataset in self.datasets]
        class_occurances_index = [np.array(dataset.data.groupby("label").count()["id"].index) for dataset in self.datasets]
        class_occurance_barplot = barplot(counts=class_occurances_values, values=class_occurances_index, bar_type="vertical", height=(self.height//5)*2, width=floor(self.width/2))

        dublication_data = {dataset.name if dataset.name is not None else "Dataset_"+str(index): [getattr(dataset, self.DESCRIPTOR_DATA).duplicated().sum()] for index, dataset in enumerate(self.datasets)}
        dublication_data["All"] = pd.concat(self._get_descriptor_for_all_datasets(self.DESCRIPTOR_DATA)).duplicated().sum()
        dublication_df = pd.DataFrame(dublication_data)
        dublication_overview = table_from_dataframe(dublication_df)
        
        return pn.Column(
            "<b>Dublications</p>", pn.Row(dublication_overview), 
            "<b>Dataset stats</b>", pn.Row(*dataset_overview_table), 
            "<b>Image stats</b>", pn.Row(*images_overview_table), 
            "<b>Class stats</b>", pn.Row(*classes_overview_table), 
            pn.Row(*class_occurance_barplot, align="center")
        )
    
    def _generate_annotations_tab(self):
        plot_size = min(floor(self.width/len(self.datasets)), floor(self.height/2))
        link_plots_checkbox = pnw.Checkbox(name="Link plot axis", value=True)
        
        @pn.depends(link_plots_checkbox.param.value)
        def _mixing_plots(link_plots):
            # mixing of classes
            mixing_matrix_classes_in_images = [utils.calculate_mixing_matrix(dataset, self.IMAGE_IDENTIFIER_COL, self.ANNOTATON_LABEL_COL) for dataset in self._get_descriptor_for_all_datasets(self.DESCRIPTOR_DATA)]
            class_mixing_matrix_plot = pn.Row("<b>Class mixing</b>", *heatmap(mixing_matrix_classes_in_images, "row_name", "col_name", "values", link_plots=link_plots, width=plot_size, height=plot_size))
            # number of object per image, stacked hist
            classes_for_objects_per_image_stacked_hist = pn.Row(
                "<b>Objects per Image</b>", 
                *stacked_hist(self._get_descriptor_for_all_datasets(self.DESCRIPTOR_DATA), self.OBJECTS_PER_IMAGE_COL, self.ANNOTATON_LABEL_COL, "Objects per Image", link_plots=link_plots, width=plot_size, height=plot_size)
            )
            return pn.Column(link_plots_checkbox, class_mixing_matrix_plot, classes_for_objects_per_image_stacked_hist)
            
        # categorical overview
        self.categorical_2d_histogram = categorical_2d_histogram_with_gui(
            self._get_descriptor_for_all_datasets(self.DESCRIPTOR_DATA),
            category_cols=["label", "num_annotations", "width", "height"],
            hist_cols=["num_annotations", "area", "area_normalized", "bbox_ratio", "bbox_xmin", "bbox_xmax", "bbox_ymin", "bbox_ymax", "width", "height"],
            height=floor(plot_size*1.5), width=floor(plot_size*1.5)
        )
        return pn.Column(_mixing_plots, self.categorical_2d_histogram, align="center")
    
    def _generate_gallery_tab(self):
        return pn.Row(*[Gallery(dataset, "data", "filepath", ["num_annotations", "width", "height", "label", "area", "bbox_ratio", "bbox_width", "bbox_height"], width=floor(self.width/len(self.datasets))).show() for dataset in self.datasets], align="start", sizing_mode="stretch_both")
    
    def build_gui(self):
        dataset_tab = self._generate_dataset_tab()
        dataset_stats_tab = self._generate_datset_stats_tab()
        annotations_tab = self._generate_annotations_tab()
        gallery_tab = self._generate_gallery_tab()
        self.gui = pn.Tabs(("Dataset stats overview", dataset_stats_tab), ("Annotations overivew", annotations_tab), ("Gallery", gallery_tab), ("Dataset overview", dataset_tab), align="start")

In [ ]:
test_object_detection_comparison = ObjectDetectionDatasetComparison([test_valid_record_dataset, test_train_record_dataset], width=1700, height=700)
test_object_detection_comparison.show()

In [ ]:
#export
class ObjectDetectionDatasetGenerator(DatasetGenerator):
    DESCRIPTOR_STATS = "stats_dataset"
    DATASET_OVERVIEW = ObjectDetectionDatasetOverview
    DATASET_FILTER_COLUMNS = ["width", "height", "label", "area_normalized", "bbox_ratio", "bbox_width", "bbox_height", "num_annotations"]

In [ ]:
test_dataset_generator = ObjectDetectionDatasetGenerator(test_train_record_dataset, height=1000, width=1500)
test_dataset_generator.show()

In [ ]:
#export
class ObjectDetectionResultOverview(Dashboard):
    def __init__(self, dataset):
        self.dataset= dataset
        super().__init__()
        
    def build_gui(self):
        loss_tab = self.build_loss_tab()
        
        self.gui = pn.Tabs(("Loss", loss_tab))
        # self.gui = None
    
    def show(self):
        return self.gui
    
    def build_loss_tab(self):
        # loss hists
        bins_input = pnw.IntInput(name="Bins", start=1, end=100, value=10)

        @pn.depends(bins_input.param.value)
        def loss_hists(bins):
            unique_losses = odrd.base_data[["filepath", "loss_classifier", "loss_box_reg", "loss_objectness", "loss_rpn_box_reg", "loss_total"]].drop_duplicates()
            hist_line = plots_as_matrix(
                histogram(
                    [unique_losses[loss] for loss in ["loss_total", "loss_classifier", "loss_box_reg", "loss_objectness", "loss_rpn_box_reg"]],
                    title=["loss_total", "loss_classifier", "loss_box_reg", "loss_objectness", "loss_rpn_box_reg"], 
                    bins=bins, linked_axis=False), 5, 1, width=1500, height=200
            )
            return hist_line
        pn.Column(bins_input, loss_hists)
        
        axis_cols = ['score', 'area_normalized', 'area', 'bbox_ratio', 'bbox_width', 'bbox_height', 'num_annotations', 'loss_classifier', 'loss_box_reg', 'loss_objectness', 'loss_rpn_box_reg', 'loss_total', 'width', 'height']
        scatter_overview = scatter_plot_with_gui(
            odrd.base_data[odrd.base_data["is_prediction"] == True], 
            x_cols=axis_cols[1:] + [axis_cols[0]],
            y_cols=axis_cols,
            color_cols=["label", "num_annotations", "filename"]
        )
        
        cat_2d_hist = categorical_2d_histogram_with_gui(
            odrd.base_data[odrd.base_data["is_prediction"] == True],
            category_cols=["label", "num_annotations", "filename"], 
            hist_cols=['loss_total', 'loss_classifier', 'loss_box_reg', 'loss_objectness', 'loss_rpn_box_reg', 'score', 'area_normalized', 'area', 'bbox_ratio', 'bbox_width', 'bbox_height', 'num_annotations', 'width', 'height', 'label']
        )
        
        sub_tabs = pn.Tabs(
            ("Histograms", pn.Row(pn.Spacer(sizing_mode="stretch_width"), scatter_overview, pn.Spacer(sizing_mode="stretch_width"), cat_2d_hist, pn.Spacer(sizing_mode="stretch_width"), align="center")),
            ("Gallery", Gallery(self.dataset, "base_data", "filepath", sort_cols=["loss_total", "loss_classifier", "loss_box_reg", "loss_objectness", "loss_rpn_box_reg"], height=self.height).show())
        )
        
        return pn.Column(pn.Column(bins_input, loss_hists), sub_tabs)

In [ ]:
odrd = ObjectDetectionResultsDataset.load("test_results/pets_valid.dat")
odrdash = ObjectDetectionResultOverview(odrd)

In [ ]:
odrdash.show()

In [ ]:
Gallery(odrdash.dataset, "base_data", "filepath")